End to end machine learning workflow to predict mortality within 180 days of last visit


This is step 4

We will populate the conditions for each patient in our training set

# Import Packages and Data

In [18]:
# Package Import

import pandas as pd

In [19]:
# import data

demographics = pd.read_csv('output/step_3_train_demographics.csv')
conditions = pd.read_csv('input/condition_occurrence.csv')

/var/folders/12/24w9w2w944x4_v3m36w_mnth0000gn/T/ipykernel_59721/4220432051.py:4: DtypeWarning: Columns (2,3,7,8,13) have mixed types. Specify dtype option on import or set low_memory=False.
  conditions = pd.read_csv('input/condition_occurrence.csv')


# Populate Condition Features

In [20]:
# Preview demographics

demographics.head()

,person_id,outcome,age_Charlson_index,gender_source_value,ethnicity_source_value,age_last_visit
0,138730,1,3,1,1,28.0
1,127727,1,6,1,2,65.0
2,46210,1,2,1,3,69.0
3,10249,1,4,2,2,80.0
4,78014,1,3,1,1,79.0


In [21]:
# Preview conditions

conditions.head()

,provider_id,visit_detail_id,condition_start_datetime,condition_end_datetime,condition_source_concept_id,person_id,condition_status_source_value,condition_end_date,condition_start_date,condition_status_concept_id,condition_type_concept_id,condition_concept_id,stop_reason,condition_source_value,condition_occurrence_id,visit_occurrence_id
0,NaN,NaN,2008-11-20,NaN,44829639.0,115208.0,NaN,NaN,2008-11-20,4230359.0,38000230.0,312922.0,NaN,V4581,9070047.0,NaN
1,NaN,NaN,2009-02-18,NaN,44825447.0,115208.0,NaN,NaN,2009-02-18,4230359.0,38000230.0,43020432.0,NaN,44021,5258637.0,NaN
2,NaN,NaN,2008-08-24,NaN,44828074.0,115208.0,NaN,NaN,2008-08-24,NaN,38000230.0,75354.0,NaN,7329,12839271.0,NaN
3,NaN,NaN,2010-05-14,NaN,44830080.0,115208.0,NaN,NaN,2010-05-14,4230359.0,38000230.0,42872402.0,NaN,41401,4019921.0,2042432.0
4,NaN,NaN,2009-02-08,NaN,44828981.0,115208.0,NaN,NaN,2009-02-08,4230359.0,38000230.0,316999.0,NaN,4279,4022174.0,NaN


In [22]:
# Select the features we need from conditions

conditions = conditions[['person_id','condition_concept_id']]
conditions = conditions.reset_index(drop=True)

In [23]:
# create a list of each patient

patients = list(demographics['person_id'])

In [24]:
# filter conditions to only include conditions from the patients we have

conditions = conditions.loc[conditions.person_id.isin(patients)]
conditions = conditions.reset_index(drop=True)

In [25]:
# Check how many NA rows in conditions

len(conditions.loc[conditions.condition_concept_id.isna()])

conditions = conditions.loc[pd.notna(conditions['condition_concept_id'])]
conditions = conditions.reset_index(drop=True)

In [27]:
# Create a list of string condition concept ids for the columns
# use set to avoid duplicates

condition_names = list(set(conditions['condition_concept_id']))

In [28]:
# Transform the condition names into strings

storage_list = []
for i in range(len(condition_names)):
    storage_list.append(str(condition_names[i]))
    
conditions_names = storage_list

In [29]:
# Populate a df with initialized values as 0

columns = len(conditions_names)
rows = len(patients)


populated_conditions = []

for i in range(rows):
    current_row = []
    for j in range(columns):
        current_row.append(0)
    populated_conditions.append(current_row)
    
populated_conditions = pd.DataFrame(populated_conditions,columns = conditions_names)

populated_conditions.insert(loc=0, column='person_id', value=patients)

In [31]:
# Preview created df

populated_conditions.head()

,person_id,0.0,196620.0,196628.0,196645.0,196654.0,196726.0,196733.0,196734.0,196735.0,...,196461.0,196464.0,196468.0,196469.0,196471.0,196476.0,40304526.0,196523.0,4161462.0,4063171.0
0,138730,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,127727,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,46210,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10249,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,78014,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# We Populate our df by going through each row in conditions 
# Conditions has two columns: person_id and condition_concept_id
# We store the current person and the current concept_id
# We then loop through the patients list and find the index of the person we want
# Then we just reference the df location by matching the person index and string of concept id


for i in range(len(conditions)):
    current_person = conditions.loc[i][0]
    concept_id = conditions.loc[i][1]

    for j in range(len(patients)):
        if(patients[j] == current_person):
            person_index = j

    populated_conditions.loc[person_index][str(concept_id)] = 1

In [35]:
# Preview populated conditions

populated_conditions.tail()

,person_id,0.0,196620.0,196628.0,196645.0,196654.0,196726.0,196733.0,196734.0,196735.0,...,196461.0,196464.0,196468.0,196469.0,196471.0,196476.0,40304526.0,196523.0,4161462.0,4063171.0
9995,19286,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,131545,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,128576,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9998,21276,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9999,87671,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# Join to demographics

demographics = demographics.join(populated_conditions.set_index('person_id'), on='person_id')

In [37]:
demographics

,person_id,outcome,age_Charlson_index,gender_source_value,ethnicity_source_value,age_last_visit,0.0,196620.0,196628.0,196645.0,...,196461.0,196464.0,196468.0,196469.0,196471.0,196476.0,40304526.0,196523.0,4161462.0,4063171.0
0,138730,1,3,1,1,28.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,127727,1,6,1,2,65.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,46210,1,2,1,3,69.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10249,1,4,2,2,80.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,78014,1,3,1,1,79.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19286,0,10,1,3,75.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,131545,0,3,2,1,75.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,128576,0,12,2,1,81.0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9998,21276,0,7,1,1,75.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# Export Data

In [38]:
demographics.to_csv('output/step_4_train_demographics.csv',index=False)